In [1]:
import json
import pandas as pd

# Laden der Liste von Dateipfaden aus der JSON-Datei
with open('results/file_paths.json', 'r') as file:
    file_paths = json.load(file)

# Laden der DataFrames aus den Pickle-Dateien für categorized_medical_reports
categorized_medical_reports = []
for i in range(len(file_paths)):  # Hier nehmen wir an, dass die Anzahl der Berichte der Anzahl der Dateipfade entspricht
    df = pd.read_pickle(f'results/categorized_medical_reports/categorized_report_{i}.pkl')
    categorized_medical_reports.append(df)

# Laden der DataFrames aus den Pickle-Dateien für outcomes_of_medical_reports
outcomes_of_medical_reports = []
for i in range(len(file_paths)):  # Hier nehmen wir an, dass die Anzahl der Berichte der Anzahl der Dateipfade entspricht
    df = pd.read_pickle(f'results/outcomes_of_medical_reports/outcomes_report_{i}.pkl')
    outcomes_of_medical_reports.append(df)

# Laden der Liste von extrahierten Texten aus der JSON-Datei
with open('results/extracted_texts.json', 'r') as file:
    extracted_texts = json.load(file)

# Laden der Liste von zusammengefassten vereinfachten medizinischen Berichten aus der JSON-Datei
with open('results/summarized_simplified_medical_reports.json', 'r') as file:
    summarized_simplified_medical_reports = json.load(file)

In [4]:
outcomes_of_medical_reports[0]

,Stichwort,Modellvorhersage
0,Weitere Behandlung notwendig,unklar
1,Komplikationen aufgetreten,nein
2,Gesamtzustand des Patienten,Der Patient befindet sich in einem stabilen Zu...


In [25]:
import dash
from dash.dependencies import Input, Output
from dash import dcc, html, dash_table
import base64
import threading
import webbrowser


app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id='file-dropdown',
        options=[{'label': filepath, 'value': i} for i, filepath in enumerate(file_paths)],
        value=0  # Standardmäßig wird das erste Element ausgewählt
    ),
    # Zwischenüberschrift für den Outcomes-Bericht
    html.H3('Übersicht klinische Ergebnisse (gpt-4-0125-preview)', className='header'),
    html.Div(id='display-outcomes-report', className='content-area'),
    
    html.H3('Zusammenfassung (gpt-4-0125-preview)', className='header'),
    html.Div(id='display-summarized-report', className='content-area', style={'color': '#C711FF'}),
    
    html.H3('Kategorisierter Bericht (gpt-3.5-turbo-0125)', className='header'),
    html.Div(id='display-categorized-report', className='content-area'),
    
    html.H3('Extrahierter Text (pytesseract)', className='header'),
    html.Div(id='display-extracted-text', className='content-area', style={'color':'#08A81A'}),
        
    html.H3('Originaldatei', className='header'),
    html.Div(id='file-display'),
], className='flex-container')

@app.callback(
    Output('file-display', 'children'),
    [Input('file-dropdown', 'value')]
)
def update_file_display(selected_index):
    return display_file(selected_index)

@app.callback(
    [
        Output('display-extracted-text', 'children'),
        Output('display-categorized-report', 'children'),
        Output('display-outcomes-report', 'children'),
        Output('display-summarized-report', 'children'),
    ],
    [Input('file-dropdown', 'value')]
)

def update_text_outputs(selected_index):
    if selected_index is not None:
        # Extrahierter Text (String)
        extracted_text = html.Pre(extracted_texts[selected_index])
        
        # Kategorisierter Bericht (DataFrame)
        categorized_report_df = categorized_medical_reports[selected_index]
        categorized_report = dash_table.DataTable(
            data=categorized_report_df.to_dict('records'),
            columns=[{'name': i, 'id': i} for i in categorized_report_df.columns],
            style_table={'overflowX': 'auto', 'marginLeft': '0px'},
            style_cell={'whiteSpace': 'normal', 'height': 'auto', 'textAlign': 'left'},
            style_data_conditional=[
                {
                    'if': {
                        'column_id': 'Textstelle',
                    },
                    'color': '#119AFF',
                }
        ]
        )
        
        # Outcomes Bericht (DataFrame)
        outcomes_report_df = outcomes_of_medical_reports[selected_index]
        outcomes_report = dash_table.DataTable(
            data=outcomes_report_df.to_dict('records'),
            columns=[{'name': i, 'id': i} for i in outcomes_report_df.columns],
            style_table={'overflowX': 'auto', 'marginLeft': '0px'},
            style_cell={'whiteSpace': 'normal', 'height': 'auto','textAlign': 'left'},
            style_data_conditional=[
                {
                    'if': {
                        'column_id': 'Modellvorhersage',
                        'filter_query': '{Modellvorhersage} eq "nein"'
                    },
                    'backgroundColor': '#9BC53D',
                    'color': 'white',
                },
                {
                    'if': {
                        'column_id': 'Modellvorhersage',
                        'filter_query': '{Modellvorhersage} eq "ja"'
                    },
                    'backgroundColor': '#E55934',
                    'color': 'white',
                },
                {
                    'if': {
                        'column_id': 'Modellvorhersage',
                        'filter_query': '{Modellvorhersage} eq "unklar"'
                    },
                    'backgroundColor': '#FAE588',
                    'color': 'black',
                },
                {
                    'if': {
                        'row_index': 2,
                        'column_id': 'Modellvorhersage'
                    },
                    'backgroundColor': 'white',
                    'color': '#C711FF'
                }
            ]
        )
        
        # Zusammengefasster Bericht (String)
        summarized_report = html.Pre(summarized_simplified_medical_reports[selected_index])

        return extracted_text, categorized_report, outcomes_report, summarized_report

    # Wenn kein Index ausgewählt ist, leere Komponenten zurückgeben
    return html.Div(), html.Div(), html.Div(), html.Div()

def update_text_outputs(selected_index):
    if selected_index is not None:
        # Die restlichen Ergebnisse darstellen
        extracted_text = extracted_texts[selected_index]
        categorized_report = categorized_medical_reports[selected_index]
        outcomes_report = outcomes_of_medical_reports[selected_index]
        summarized_report = summarized_simplified_medical_reports[selected_index]

        return (
            html.Pre(extracted_text),  # Beispiel mit Pre-Formatierung
            html.Pre(categorized_report),  # Beispiel mit Pre-Formatierung
            html.Pre(outcomes_report),  # Beispiel mit Pre-Formatierung
            html.Pre(summarized_report),  # Beispiel mit Pre-Formatierung
        )
    return (html.Div(), html.Div(), html.Div(), html.Div())

def display_file(selected_index):
    file_path = file_paths[selected_index]
    if file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
        encoded_image = encode_image(file_path)
        return html.Img(src=encoded_image)
    elif file_path.lower().endswith('.pdf'):
        encoded_pdf = encode_pdf(file_path)
        return html.Iframe(src=encoded_pdf, style={'height': '500px', 'width': '100%'})
    return "Dateiformat wird nicht unterstützt."

def encode_image(image_path):
    with open(image_path, 'rb') as image_file:
        encoded = base64.b64encode(image_file.read()).decode('utf-8')
    return f'data:image/png;base64,{encoded}'

def encode_pdf(pdf_path):
    with open(pdf_path, 'rb') as pdf_file:
        encoded = base64.b64encode(pdf_file.read()).decode('utf-8')
    return f'data:application/pdf;base64,{encoded}'

def open_browser():
    webbrowser.open('http://127.0.0.1:8050/')

if __name__ == '__main__':
    # Threading, um den Browser zu öffnen, während der Server läuft
    threading.Timer(1.5, open_browser).start()
    app.run_server(debug=True)